In [2]:
import pandas as pd
import numpy as np

# import the master dataset that SF-Fire-Risk is using
masterdf = pd.read_csv('data/masterdf_20170920.csv', encoding='ISO-8859-1', low_memory=False) #, 
                      #usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23])

# usecols=[1, 2, 3, 6,7,], 

In [3]:
# need to also merge in the census tracts from another csv file (oops should've used that originally!)
from column_report import get_column_report
import os
import sys
# to import my column_report package
mod_path = os.path.abspath(os.path.join('..'))
if mod_path not in sys.path:
    sys.path.append(mod_path)
    


master_tracts = pd.read_csv('data/masterdf_inc_census_tract.csv', encoding='ISO-8859-1',low_memory=False,
                           usecols=[3,34])

# for idx, col in enumerate(master_tracts.columns):
#     print(idx, col)

# to ensure we can merge without duplicates    
master_tracts = master_tracts.drop_duplicates()
masterdf = masterdf.drop_duplicates()


master_tracts = master_tracts[master_tracts.EAS.isin(masterdf.EAS)]

masterdf = masterdf.merge(master_tracts, on=['EAS'], how='inner')

print(len(masterdf))
   


195308


In [4]:
# get rid of spaces in column names
cols = []
for c in masterdf.columns:
    cols.append(c.replace(' ','_'))
    
masterdf.columns = cols

# let's get the columns and their indices
for idx, col in enumerate(masterdf.columns):
    print(idx, col)

#masterdf.head(3)

0 Unnamed:_0
1 Incident_Date
2 EAS
3 Incident_Year
4 Incident_Cat
5 Incident_Dummy
6 Neighborhood
7 Location_y
8 Address
9 Building_Cat
10 Yr_Property_Built
11 Num_Bathrooms
12 Num_Bedrooms
13 Num_Rooms
14 Num_Stories
15 Num_Units
16 Perc_Ownership
17 Land_Value
18 Property_Area
19 Assessed_Improvement_Val
20 Tot_Rooms
21 landval_psqft
22 count_potential_fire_control
23 count_all_complaints
24 count_all_complaints_not_corrected
25 count_potential_fire_control_not_corrected
26 count_fire_emergency_safety
27 count_potential_fire_cause
28 count_fire_emergency_safety_not_corrected
29 count_potential_fire_cause_not_corrected
30 tract


In [5]:
# get an idea of what we're dealing with here

print('dimensions of the dataset: ', masterdf.shape)

# oooooh! Incident_Dummy is our label! 
print('number of fire incidents: ', len(masterdf[masterdf.Incident_Dummy==1]))

# each building in SF has an EAS code
print('number of EAS codes (buildings): ', len(masterdf.EAS.unique()))

dimensions of the dataset:  (195308, 31)
number of fire incidents:  24791
number of EAS codes (buildings):  182644


In [6]:
# get a snippet of the dataframe (minus the count_all* columns that I think are useless)
masterdf.drop(masterdf.columns[-11:-3], axis=1).head()

,Unnamed:_0,Incident_Date,EAS,Incident_Year,Incident_Cat,Incident_Dummy,Neighborhood,Location_y,Address,Building_Cat,...,Num_Rooms,Num_Stories,Num_Units,Perc_Ownership,Land_Value,Property_Area,Assessed_Improvement_Val,count_fire_emergency_safety_not_corrected,count_potential_fire_cause_not_corrected,tract
0,0,2015-06-20,451005,2015.0,COOKING FIRE,1.0,SUNSET/PARKSIDE,"(37.7543289339354, -122.480327187833)",1532 NORIEGA ST,COMMERCIAL USE,...,11.0,3.0,2.777778,1.0,4.384343e+05,4135.0,262181.666667,0.0,0.0,32802
1,1,2010-11-28,360149,2010.0,COOKING FIRE,1.0,MISSION,"(37.7645472195468, -122.418358468789)",135 CAPP ST,APARTMENT,...,36.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,0.0,0.0,20100
2,2,2011-04-26,360149,2011.0,COOKING FIRE,1.0,MISSION,"(37.7645472195468, -122.418358468789)",135 CAPP ST,APARTMENT,...,36.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,0.0,0.0,20100
3,3,2006-03-09,360149,2006.0,BUILDING FIRE,1.0,MISSION,"(37.7645472195468, -122.418358468789)",135 CAPP ST,APARTMENT,...,36.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,0.0,0.0,20100
4,4,2004-05-28,360149,2004.0,OUTDOOR FIRE,1.0,MISSION,"(37.7645472195468, -122.418358468789)",135 CAPP ST,APARTMENT,...,36.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,0.0,0.0,20100


In [7]:
#### SETUP ####
# match our data points with their census blocks so we can incorporate AHS data!
#!pip install censusgeocode
# package from US Census Geocoder API
from censusgeocode import CensusGeocode

cg = CensusGeocode()

# need to format Location_y differently in order to input to censusgeocoder
def split_coords(inputstring): 
    coords = inputstring.strip('()').split(',')
    coords = [float(i) for i in coords]
    return coords

masterdf['latlong'] = masterdf['Location_y'].apply(split_coords)
masterdf = masterdf.drop('Location_y', axis=1)

# function to get the census block from the geo coordinates
def get_block(coords):
    res = cg.coordinates(x=coords[1],y=coords[0])
    block = res[0].get('2010 Census Blocks')[0].get('BLOCK')
    print(block) # so we can see its progress
    return block

# function to split up masterdf.latlong into subsets
def split_seq(seq, size):
    newseq = []
    splitsize = 1.0/size*len(seq)
    for i in range(size):
        newseq.append(seq[int(round(i*splitsize)):int(round((i+1)*splitsize))])
    return newseq

# gonna try splitting it up and running one little chunk at a time and saving it
coords_subsets = split_seq(masterdf.latlong, 100)



In [13]:
# # blocks that we've already calculated: (need to add them to master)
%store -r block_subset

# to get rid of duplicate entries (overlapping indices)
block_subset = block_subset.groupby(block_subset.index).first()

# now add the blocks to the masterdf
masterdf['census_block'] = block_subset
print(len(masterdf))

# ahh.. solution to the unhashable list errors!
masterdf['latlong'] = masterdf['latlong'].apply(tuple)

masterdf.columns
#print(len(coords_subsets))

In [24]:
################################ working with census geocoder API ######################################
####### don't need to run this cell again (already filled and stored census blocks for each row) #######

# had to keep re-running the cg.coordinates() so this cell 
# gets a list (of the indicies) of where census_block is still null
stillneed = pd.Series(masterdf[masterdf['census_block'].isnull()].index)
print(len(stillneed))
# then filter through coords_subsets and remove those we've already ran

# create a separate list with completed subsets removed
coords_stillneed = []


for i, coords in enumerate(coords_subsets):
    # create a series of the indices within current subset
    print(i)
    c = pd.Series(coords.index)
    print(c.isin(stillneed).all())
    # check if we still need the block for these coordinates
    # then we can get rid of it
    # so, if the whole set is contained in the stillneed series
    # (can't use .any() because stillneed also includes blocks we've checked
    # but gave null, so every coordssubset will return true)
    if c.isin(stillneed).all():
        coords_stillneed.append(coords)
        #print(coords_stillneed[i][:3])
        print('keeping subset ', i)


        
# loops through and finds the census block for each row
# divided the whole dataset into 100 subsets, computing blocks for each subset and then storing them
for i in range(len(coords_stillneed)):
    lilblocks = coords_stillneed[i].apply(get_block)
    block_subset = block_subset.append(lilblocks)
    %store block_subset
    print(len(block_subset), ' blocks')
    # to see how many subsets are remaining:
    print('-'*200,'\n',len(coords_stillneed) - i,' bottles of beer on the wall')

In [17]:
print('number of unique combinations of (census_block, tract) in the master dataset: %s' 
      %len(masterdf.groupby(['census_block', 'tract']).size()))

# now let's take a closer look at our df!
get_column_report(masterdf)

# count the number of buildings in each block
#print(masterdf[['EAS', 'census_block', 'tract']].groupby(['census_block', 'tract']).agg(['count']).reset_index())


number of unique combinations of (census_block, tract) in the master dataset: 4981


,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
29,tract,int64,192,32802,0,0.00
2,EAS,float64,182644,451005,0,0.00
20,landval_psqft,float64,137246,106.03,0,0.00
17,Property_Area,float64,18303,4135,0,0.00
16,Land_Value,float64,80330,438434,0,0.00
6,Neighborhood,object,39,SUNSET/PARKSIDE,0,0.00
7,Address,object,182603,1532 NORIEGA ST,0,0.00
8,Building_Cat,object,8,COMMERCIAL USE,0,0.00
9,Yr_Property_Built,float64,529,1989,0,0.00
30,latlong,object,144203,"(37.7543289339354, -122.480327187833)",0,0.00


In [25]:
##### DEALING WITH MISSING VALUES ######

# first let's deal with missing block values
masterdf.census_block = pd.to_numeric(masterdf.census_block)

# replace missing vals with the block in the following row (they're usually grouped together)
masterdf.census_block = masterdf.census_block.fillna(0).replace(0,method='bfill').astype(int)

masterdf.census_block.head() #= masterdf.census_block.apply(lambda x: int(x))

# # these cols have mostly zeros and prob won't be useful
# masterdf = masterdf.drop(['Num_Bedrooms', 'num_complaints'], axis=1)

len(masterdf[masterdf.census_block==0])


0

In [31]:
# now dealing with missing vals in num_rooms, tot_rooms
# there are fewer tot_rooms zero vals but otherwise they're pretty much always zero in the same rows
# so using tot_rooms as an indicator of values we want to fill for simplicity
# also I don't know what the difference between those two categories are


print('Number of zero vals for Tot_Rooms in the masterdf: %i ' % len(masterdf[masterdf.Tot_Rooms==0]))
print('Number of unique census tracts that contain these zero vals: %i \n' \
          % len(masterdf['tract'][masterdf.Tot_Rooms==0].unique()))

# so the idea is to check the percentage of missing values in each of the tracts that have zeros
# and if less than half of the tract's data is missing, then we can impute those values with the tract medians
# and otherwise we just delete the row (there arent many)

# print a report of what the tracts look like:

def tractreport(tract_list):
    tract_rep = []
    for t in tract_list:
        minidf = masterdf[masterdf.tract==t]
        num_nan = len(minidf[minidf.Tot_Rooms==0])
        pct_nan = 100*num_nan/float(len(minidf))
        totrooms_median = minidf.Tot_Rooms[minidf.Tot_Rooms!=0].median()
        numrooms_median = minidf.Num_Rooms[minidf.Num_Rooms!=0].median()
#         print('-'*75, '\ntract: %s' % str(t),  '\nnumber of nulls: %i' % num_nan, 
#                 '\ntotal number of blocks in the tract: %i' % len(minidf),
#                 '\n%2.2f percent of blocks have missing info' % pct_nan,
#                 '\nmedian total rooms: %i' % totrooms_median, 
#                 '\nmedian num rooms: %i' % numrooms_median)
        tract_rep.append([t, pct_nan, totrooms_median, numrooms_median])
    cols=['tract', 'pct_nan', 'median_totrooms', 'median_numrooms']
    tract_rep = pd.DataFrame(tract_rep, columns=cols)
    return tract_rep
        
        
tract_report = tractreport(masterdf['tract'][masterdf.Tot_Rooms==0].unique())

tract_report.head(10)

Number of zero vals for Tot_Rooms in the masterdf: 3618 
Number of unique census tracts that contain these zero vals: 162 



,tract,pct_nan,median_totrooms,median_numrooms
0,12501,19.911504,60.000000,60.0
1,60700,29.665072,8.470446,6.0
2,16000,24.031008,9.000000,9.0
3,17601,14.577259,15.000000,15.0
4,12502,29.813665,59.000000,57.0
5,11700,35.243997,18.000000,18.0
6,22704,10.211592,9.000000,7.0
7,17801,22.448980,7.871429,6.0
8,22801,3.861004,11.000000,10.0
9,61500,23.969631,10.285714,10.0


In [27]:
# now we loop through and fill 'em!
cols_tofill = {'Tot_Rooms':'median_totrooms', 'Num_Rooms':'median_numrooms'}

filled_master = masterdf.reset_index().copy()


for row in masterdf.reset_index().iterrows():
    tract = row[1]['tract'] 
    # check if this row specifically is missing vals
    # as long as more than 75% arent missing from the tract, fill with tract median
    # (tract in list(tract_report.tract)) &
    if row[1]['Tot_Rooms']==0:
        report = tract_report[tract_report.tract==tract]
        if report['pct_nan'].item() < 75.0:
            for c in cols_tofill:
                filled_master.loc[row[0], c] = report[cols_tofill[c]].item()
        else:
            filled_master = filled_master.drop(row[0])

# maybe also want to calculte number of fire incidents in each block that we're dropping? idk            
print('%i rows dropped' % (len(masterdf) - len(filled_master)))         

36 rows dropped


In [28]:
filled_master.drop(['Incident_Year', 'Incident_Date', 'Incident_Cat', 'Num_Bedrooms', 'Num_Bathrooms'], 
                  axis=1, inplace=True)

get_column_report(filled_master)

,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
13,Property_Area,float64,18301,4135,0,0.00
25,tract,int64,192,32802,0,0.00
16,landval_psqft,float64,137244,106.03,0,0.00
15,Tot_Rooms,float64,1349,11,0,0.00
26,latlong,object,144198,"(37.7543289339354, -122.480327187833)",0,0.00
12,Land_Value,float64,80328,438434,0,0.00
9,Num_Stories,float64,343,3,0,0.00
27,census_block,int64,287,3000,0,0.00
6,Building_Cat,object,8,COMMERCIAL USE,0,0.00
5,Address,object,182598,1532 NORIEGA ST,0,0.00


In [107]:
# maybe - check correlation between num_rooms & tot_rooms so maybe can
# just drop num_rooms
# probably want to impute the num_units
# and then think about the children data just downloaded from IPUMS
# then drop the cols we don't want so we can get a cleaned 'n filled
# dataset to run the model

0


In [23]:
# using Random Forest regressor to impute missing vals

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# impute missing values in Assessed_Improvement_Val column
# creating new dataframe filled_master_clean so we don't mess with filled_master

filled_master_clean = filled_master.copy()
filled_master_clean.drop(['index', 'Unnamed:_0'], axis=1, inplace=True)
# to get only numerical columns
filled_master_clean = filled_master_clean[filled_master_clean.corr().columns]

X = filled_master_clean.drop('Assessed_Improvement_Val', axis=1)
y = filled_master_clean['Assessed_Improvement_Val']

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=15, test_size=0.2)

clf = RandomForestRegressor()

# first step, we want to make sure the rf regressor performs well for this column
clf.fit(x_train, y_train)

scores = cross_val_score(clf, x_train, y_train, cv=5)
print('mean cross val score: %.4f' % scores.mean())

print('R^2 for the testing dataset: %.4f ' % clf.score(x_test, y_test))

# we get a very good R^2, so feeling good about using this method to impute values


# grab the missing vals that we want to fill in our master dataset:
x_tofill = filled_master_clean[filled_master_clean.Assessed_Improvement_Val==0].drop('Assessed_Improvement_Val', axis=1)

# and we'll want to re-train the regressor with the non-empty data
fulls = [i for i in filled_master_clean.index if i not in x_tofill.index]

x_full = X.loc[fulls]
y_full = y.loc[fulls]

clf.fit(x_full, y_full)


# and then we want to use regressor to fill the values for filled_master that we were missing 
# later can go back and merge it into filled_master?? 
# 
filled_master_clean['Assessed_Improvement_Val'].loc[x_tofill.index] = \
        pd.Series(clf.predict(x_tofill), index=x_tofill.index)

# really need someone to tell me about the settingwcopywarnings

mean cross val score: 0.9528
R^2 for the testing dataset: 0.9734 


In [32]:
# Filling Num_Units missing values! 
# reassigning X and y and the other vars because we don't need them for assessed_improvement_.. anymore
# (I should have made a function where you input the column_name that you want to impute, but now 
# I don't really feel like going back and adjusting what I've already done)
X = filled_master_clean.drop(['Num_Units'], axis=1)
y = filled_master_clean['Num_Units']

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=15, test_size=0.2)

# my question about clf again.. should we make a new one??
clf = RandomForestRegressor()

clf.fit(x_train, y_train)

scores = cross_val_score(clf, x_train, y_train, cv=5)

print('mean cross val score: %.4f' % scores.mean())

print('R^2 for the testing dataset: %.4f ' % clf.score(x_test, y_test))


# the missing vals that we want to fill in our master dataset:
x_tofill = filled_master_clean[filled_master_clean.Num_Units==0].drop('Num_Units', axis=1)

# and we'll want to re-train the regressor with the non-empty data
fulls = [i for i in filled_master_clean.index if i not in x_tofill.index]

x_full = X.loc[fulls]
y_full = y.loc[fulls]

clf.fit(x_full, y_full)


# and then we want to use regressor to fill the values for filled_master that we were missing 
# later can go back and merge it into filled_master?? 
# 
filled_master_clean['Num_Units'].loc[x_tofill.index] = \
        pd.Series(clf.predict(x_tofill), index=x_tofill.index)

mean cross val score: 0.9443
R^2 for the testing dataset: 0.9550 


In [33]:
# I'm guessing that Num_Rooms is very similar to Tot_Rooms and that we should just drop that column
print(filled_master.Num_Rooms.corr(filled_master.Tot_Rooms))
# yep. feel OK about deleting it. That means the only one we still *might* want to fill values is Num_Units
# and it's probably too few rows to spend time filling them... but let's do it, for practice !

# and then I'm going to drop all of these columns because they're 99% zeros
filled_master_clean.drop(['Num_Rooms','count_potential_fire_cause','count_fire_emergency_safety',
                         'count_all_complaints_not_corrected', 'count_potential_fire_cause', 
                         'count_potential_fire_control_not_corrected', 'count_potential_fire_cause_not_corrected',
                         'count_fire_emergency_safety_not_corrected'], axis=1, inplace=True)


get_column_report(filled_master_clean)

0.926744664763


,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
0,EAS,float64,182639,451005.00,0,0.00
2,Yr_Property_Built,float64,529,1989.00,0,0.00
3,Num_Stories,float64,343,3.00,0,0.00
4,Num_Units,float64,685,2.78,0,0.00
6,Land_Value,float64,80328,438434.33,0,0.00
7,Property_Area,float64,18301,4135.00,0,0.00
8,Assessed_Improvement_Val,float64,97512,262181.67,0,0.00
9,Tot_Rooms,float64,1349,11.00,0,0.00
10,landval_psqft,float64,137244,106.03,0,0.00
13,tract,int64,192,32802.00,0,0.00


In [41]:
# we probably don't want to lose the x,y coordinates
filled_master_clean.loc[:, 'latlong'] = filled_master.latlong

filled_master_clean.head()

,EAS,Incident_Dummy,Yr_Property_Built,Num_Stories,Num_Units,Perc_Ownership,Land_Value,Property_Area,Assessed_Improvement_Val,Tot_Rooms,landval_psqft,count_potential_fire_control,count_all_complaints,tract,census_block,latlong
0,451005.0,1.0,1989.0,3.0,2.777778,1.0,4.384343e+05,4135.0,262181.666667,11.0,106.030069,0.0,0.0,32802.0,3000.0,"(37.7543289339354, -122.480327187833)"
1,360149.0,1.0,1908.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,3.0,20100.0,4000.0,"(37.7645472195468, -122.418358468789)"
2,360149.0,1.0,1908.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,3.0,20100.0,4000.0,"(37.7645472195468, -122.418358468789)"
3,360149.0,1.0,1908.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,3.0,20100.0,4000.0,"(37.7645472195468, -122.418358468789)"
4,360149.0,1.0,1908.0,3.0,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,3.0,20100.0,4000.0,"(37.7645472195468, -122.418358468789)"


In [7]:

#filled_master_clean.to_csv('data/molly_master_11132017.csv', columns=filled_master_clean.columns)

# we'll just want to store filled_master_clean and then 

In [42]:
%store filled_master_clean

Stored 'filled_master_clean' (DataFrame)
